# Lookout Audit & Insider Threat Hunting

This notebook provides threat hunting queries for investigating administrative actions and potential insider threats.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- AUDIT events ingested

## 1. Administrative Actions Overview

Review all administrative actions in the Lookout console.

In [ ]:
LookoutEvents
| where EventType == "AUDIT"
| summarize 
    ActionCount = count(),
    ActionTypes = make_set(AuditType)
    by ActorGuid, ActorType
| sort by ActionCount desc

## 2. Policy Changes

Track changes to security policies that could weaken defenses.

In [ ]:
LookoutEvents
| where EventType == "AUDIT"
| where AuditType has_any ("POLICY", "CONFIG", "SETTING")
| project 
    TimeGenerated,
    ActorGuid,
    ActorType,
    AuditType,
    AuditAttributeChanges,
    TargetType,
    TargetGuid
| sort by TimeGenerated desc

## 3. Unusual Admin Activity Patterns

Identify administrators with unusual activity volumes.

In [ ]:
LookoutEvents
| where EventType == "AUDIT"
| where ActorType in ("ADMIN_USER", "USER")
| summarize 
    DailyActions = count(),
    ActionTypes = make_set(AuditType)
    by ActorGuid, bin(TimeGenerated, 1d)
| where DailyActions > 10
| sort by DailyActions desc

## 4. Device Deactivations

Monitor device deactivation events that could indicate cleanup of compromised devices.

In [ ]:
LookoutEvents
| where EventType == "AUDIT"
| where AuditType has_any ("DEACTIVATE", "DELETE", "REMOVE")
| project 
    TimeGenerated,
    ActorGuid,
    ActorType,
    AuditType,
    TargetType,
    TargetGuid,
    TargetEmailAddress
| sort by TimeGenerated desc

## 5. Audit Activity Timeline

Visualize administrative activity over time.

In [ ]:
LookoutEvents
| where EventType == "AUDIT"
| summarize Count = count() by bin(TimeGenerated, 1h), AuditType
| render timechart

## 6. System vs User Actions

Compare automated system actions versus manual user actions.

In [ ]:
LookoutEvents
| where EventType == "AUDIT"
| summarize Count = count() by ActorType
| render piechart